<a href="https://colab.research.google.com/github/danielsoy/ALOCC-CVPR2018/blob/master/copilotiatrainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
# cd change directory?

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt

Define the Autoencoder architecture

In [ ]:
input_img = tf.keras.Input(shape=(224, 224, 3))

Encoder

In [ ]:
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
encoded = layers.MaxPooling2D((2, 2), padding='same')(x)

Decoder

In [ ]:
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(encoded)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
decoded = layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

Compile the Autoencoder with mean squared error loss

In [ ]:
autoencoder = models.Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

Prepare the dataset with data augmentation

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.5,
    #rotation_range=20,
    #width_shift_range=0.2,
    #height_shift_range=0.2,
    #shear_range=0.2,
    #zoom_range=0.2,
    #horizontal_flip=True,
    #fill_mode='nearest'
    )

Custom data generator that yields both input and target data

In [ ]:
def custom_data_generator(generator):
    for batch in generator:
        yield (batch, batch)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/datasets/train',
    target_size=(224, 224),
    batch_size=1,
    class_mode=None,  # No labels needed
    subset='training'
)

Found 75 images belonging to 1 classes.


In [ ]:
validation_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/datasets/validation',
    target_size=(224, 224),
    batch_size=1,
    class_mode=None,  # No labels needed
    subset='validation'
)

Found 75 images belonging to 1 classes.


Train the Autoencoder

In [ ]:
history = autoencoder.fit(
    custom_data_generator(train_generator),
    epochs=100,
    validation_data=custom_data_generator(validation_generator),
    steps_per_epoch=len(train_generator),
    validation_steps=len(validation_generator),
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)]
)

Epoch 1/100
75/75 [==============================] - 13s 154ms/step - loss: 0.0235 - val_loss: 0.0019
Epoch 2/100
75/75 [==============================] - 11s 152ms/step - loss: 0.0021 - val_loss: 0.0021
Epoch 3/100
75/75 [==============================] - 12s 156ms/step - loss: 0.0019 - val_loss: 0.0015
Epoch 4/100
75/75 [==============================] - 11s 152ms/step - loss: 0.0017 - val_loss: 0.0020
Epoch 5/100
75/75 [==============================] - 12s 154ms/step - loss: 0.0018 - val_loss: 0.0015
Epoch 6/100
75/75 [==============================] - 12s 154ms/step - loss: 0.0015 - val_loss: 0.0012
Epoch 7/100
75/75 [==============================] - 12s 155ms/step - loss: 0.0014 - val_loss: 0.0011
Epoch 8/100
75/75 [==============================] - 12s 155ms/step - loss: 0.0014 - val_loss: 0.0012
Epoch 9/100
75/75 [==============================] - 12s 154ms/step - loss: 0.0013 - val_loss: 0.0012
Epoch 10/100
75/75 [==============================] - 12s 155ms/step - loss: 0.001

Save the Autoencoder model

In [ ]:
autoencoder.save('anomaly_autoencoder_model.h5')